# test 特征工程
## 1.feature_edge feature_engineering feature_length
## 2.文件行数>30行的文件：截断到30行
## 3.文件行数<30行的文件：用每一个特征的均值填充到30行
## 4.如果某一个特征在某一个文件里有缺失值，则用该文件下的该特征的均值填充，如果某一个文件的某一个特征全部为缺失值，则用该文件前后两个文件的该特征的均值进行线性填充

In [2]:
# 所有图上的feature,包含feature_edge和feature_diatance,feature_length
feature_list = ['feature0', 'feature1', 'feature2', 'feature11', 'feature12', 'feature13', 'feature15', 'feature16', 'feature17', 'feature18', 
           'feature19', 'feature20_0', 'feature20_1', 'feature20_2', 'feature20_3', 'feature20_4', 'feature20_5', 'feature20_6', 
           'feature20_7','feature60', 'feature61_0', 'feature61_1', 'feature61_2', 'feature61_3',  'feature61_4', 'feature61_5', 
           'feature61_6', 'feature61_7','feature69_0', 'feature69_1', 'feature69_2', 'feature69_3','feature69_4', 'feature69_5', 
           'feature69_6', 'feature69_7', 'feature28_0', 'feature28_1', 'feature28_2','feature28_3', 'feature28_4', 'feature28_5', 'feature28_6',
       'feature28_7','feature36_0', 'feature36_1', 'feature36_2', 'feature36_3', 'feature36_4', 'feature36_5', 'feature36_6',
       'feature36_7', 'feature44_0', 'feature44_1', 'feature44_2', 'feature44_3', 'feature44_4', 'feature44_5', 'feature44_6',
       'feature44_7', 'feature52_0', 'feature52_1', 'feature52_2', 'feature52_3', 'feature52_4', 'feature52_5', 'feature52_6',
       'feature52_7', 'feature77_0', 'feature77_1', 'feature77_2', 'feature77_3',  'feature77_4', 'feature77_5', 
           'feature77_6', 'feature77_7','feature85_0', 'feature85_1', 'feature85_2', 'feature85_3','feature85_4', 'feature85_5', 
           'feature85_6', 'feature85_7', 'feature_edge', 'feature_distance','feature_length']

In [3]:
def edge_count(test):
    edge_list = [0,1,6,7,8,9,14,15,16,17,22,23,24,25,30,31]
    feature_edge = []
    for i in range(len(test)):
        current_direction = [m for m in test.iloc[i].tolist() if m in edge_list]
        feature_edge.append(len(current_direction))
    test['feature_edge'] = feature_edge
    return test.feature_edge

In [4]:
def distance_count(test):
    feature_distance = []
    for i in range(len(test)):
        coordinate = []
        for m in (test.iloc[i].tolist()):
            coordinate.append((m-8*(m//8), m//8))
        distance = 0
        for a in coordinate:
            for b in coordinate:
                distance = distance + ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5
        feature_distance.append(distance)
    test['feature_distance'] = feature_distance
    return test.feature_distance

In [5]:
look_up = pd.read_csv('D:/2022 AIops/feature engineering/new_sample_mean.csv')
look_up_column = look_up.columns.tolist()

In [13]:
train = pd.read_csv('D:/2022 AIops/feature engineering/labeled_train_mean.csv', index_col=0)
train = train.drop(columns=['1','2','3','4','5','6','index'], axis=0)
tongji = train.describe().loc[['mean','std']]
tongji

,feature0,feature1,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,...,feature69_1,feature69_2,feature69_3,feature69_4,feature69_5,feature69_6,feature_distance,feature_edge,feature_length,feature69_7
mean,188.437226,14.042381,0.406316,10.701523,329677.286138,238.642620,1362.209091,11.151503,1.784089,3.832962,...,-3.940056,-4.133553,-3.955606,-3.73240,-3.669570,-3.935985,151.073949,4.046620,10.613362,-4.137284
std,75.054971,3.196888,0.894491,4.391442,79383.220086,27.612517,286.258150,2.103542,0.677878,10.009347,...,8.542593,8.550036,8.337339,8.43978,8.592275,8.484104,31.306035,1.913684,40.711189,8.297124


In [8]:
def z_score(test):
    rows = test.index.tolist()
    columns = test.columns.tolist()
    for i in range(len(rows)):
        for j in range(len(columns)):
            test.loc[rows[i],columns[j]]=(test.loc[rows[i],columns[j]] - tongji.loc['mean',columns[j]])/tongji.loc['std',columns[j]]
    return test

In [11]:
for i in range(600):
    m = pd.read_csv('D:/2022 AIops/test/'+'{}.csv'.format(i))
    test = m[['feature20_0','feature20_1','feature20_2','feature20_3','feature20_4','feature20_5','feature20_6','feature20_7']]
    m['feature_edge'] = edge_count(test)
    m['feature_distance'] = distance_count(test)
    m['feature_length'] = [len(m)]*len(m)
    if len(m)>=30:
        m = m.iloc[:30,:]
        m = m[feature_list]
        for column in (m.columns[m.isnull().sum() > 0]).tolist():
            mean_val = m[column].mean()
            m[column].fillna(mean_val, inplace=True)
        for column in (m.columns[m.isnull().sum() > 0]).tolist():
            if column in look_up_column:
                val = look_up[look_up.index==i][column]
                m[column].fillna(val[i], inplace=True)
        m.to_csv('D:/2022 AIops/feature engineering/0120/test_30rows/{}.csv'.format(i))

    if len(m)< 30:
        m = m[feature_list]
        add = m.mean().to_frame().T
        for t in range(30-len(m)):
            m = m.append(add)        
        for column in m.columns[m.isnull().sum() > 0].tolist():
            mean_val = m[column].mean()
            m[column].fillna(mean_val, inplace=True)
        for column in (m.columns[m.isnull().sum() > 0]).tolist():
            if column in look_up_column:
                val = look_up[look_up.index==i][column]
                m[column].fillna(val[i], inplace=True)
        m.to_csv('D:/2022 AIops/feature engineering/0120/test_30rows/{}.csv'.format(i))

C:\Users\HP\AppData\Local\Temp/ipykernel_10712/3203629152.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['feature_edge'] = feature_edge
C:\Users\HP\AppData\Local\Temp/ipykernel_10712/1617231302.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['feature_distance'] = feature_distance


In [14]:
for i in range(600):
    test = pd.read_csv('D:/2022 AIops/feature engineering/0120/test_30rows/{}.csv'.format(i),index_col=0)
    test = test.reset_index()
    test = test.drop(columns=['index','feature44_0', 'feature44_1', 'feature44_2', 'feature44_3', 'feature44_4', 'feature44_5', 'feature44_6',
       'feature44_7', 'feature52_0', 'feature52_1', 'feature52_2', 'feature52_3', 'feature52_4', 'feature52_5', 'feature52_6',
       'feature52_7', 'feature77_0', 'feature77_1', 'feature77_2', 'feature77_3',  'feature77_4', 'feature77_5', 
           'feature77_6', 'feature77_7','feature85_0', 'feature85_1', 'feature85_2', 'feature85_3','feature85_4', 'feature85_5', 
           'feature85_6', 'feature85_7'], axis=1)
    test = z_score(test)
    test.to_csv('D:/2022 AIops/feature engineering/0120/test_30rows_nor/{}.csv'.format(i))